# Второе домашнее задание 
— самое большое в курсе, в нём придётся и концептуально подумать о происходящем, и технические трудности тоже порешать. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 
Третье задание — это полноценный проект по анализу данных, начиная от анализа постановки задачи и заканчивая сравнением результатов разных моделей. Задача реальная и серьёзная, хотя тему я выбрал развлекательную: мы будем строить вероятностную рейтинг-систему для спортивного “Что? Где? Когда?” (ЧГК).

Background: в спортивном “Что? Где? Когда?” соревнующиеся команды отвечают на одни и те же вопросы. После минуты обсуждения команды записывают и сдают свои ответы на карточках; побеждает тот, кто ответил на большее число вопросов. Турнир обычно состоит из нескольких десятков вопросов (обычно 36 или 45, иногда 60, больше редко). Часто бывают синхронные турниры, когда на одни и те же вопросы отвечают команды на сотнях игровых площадок по всему миру, т.е. в одном турнире могут играть сотни, а то и тысячи команд. 

Соответственно, нам нужно:
* построить рейтинг-лист, который способен нетривиально предсказывать результаты будущих турниров;
* при этом, поскольку ЧГК — это хобби, и контрактов тут никаких нет, игроки постоянно переходят из команды в команду, сильный игрок может на один турнир сесть поиграть за другую команду и т.д.; поэтому единицей рейтинг-листа должна быть не команда, а отдельный игрок;

* а что сильно упрощает задачу и переводит её в область домашних заданий на EM-алгоритм — это характер данных: начиная с какого-то момента, в базу результатов начали вносить все повопросные результаты команд, т.е. в данных будут записи вида “какая команда на какой вопрос правильно ответила”.


## Загрузка данных

In [1]:
import os
import pandas as pd
import pickle

In [2]:
os.listdir('chgk')

['results.pkl', 'players.pkl', 'tournaments.pkl']

In [3]:
with open('chgk/results.pkl', 'rb') as f:
    results_raw = pickle.load(f)
    
with open('chgk/tournaments.pkl', 'rb') as f:
    tournaments_raw = pickle.load(f)
    
with open('chgk/players.pkl', 'rb') as f:
    players_raw = pickle.load(f)

Результаты

In [4]:
results_dataframes = {k: pd.DataFrame(v) for k, v in results_raw.items()}

results = []
for k, df in results_dataframes.items():
    df['tournament_id'] = k
    results.append(df)
results = pd.concat(results, axis=0, ignore_index=True)
results.dropna(subset=['mask'], inplace=True)

In [5]:
results.head()

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,tournament_id
2407,"{'id': 1, 'name': 'Неспроста', 'town': {'id': ...",0111011101101110001101110011111111110011111100...,"{'name': 'КП - Неспроста', 'town': {'id': 201,...",67.0,None,1.0,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2408,"{'id': 2, 'name': 'Афина', 'town': {'id': 201,...",0111111101011010010101110111111111110011011111...,"{'name': 'Афина', 'town': {'id': 201, 'name': ...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2409,"{'id': 670, 'name': 'Ксеп', 'town': {'id': 201...",0011111101011010011101110011111111110111111111...,"{'name': 'Ксеп', 'town': {'id': 201, 'name': '...",65.0,None,2.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2410,"{'id': 173, 'name': 'ЮМА', 'town': {'id': 285,...",0111111001101110001101111011111111110100111111...,"{'name': 'ЮМА-Энергокапитал', 'town': {'id': 2...",64.0,None,4.5,[],[],"[{'flag': None, 'usedRating': 0, 'rating': 0, ...",22
2411,"{'id': 175, 'name': 'Транссфера', 'town': {'id...",0111111001101111001111111011111110110010111110...,"{'name': 'Транссфера', 'town': {'id': 285, 'na...",64.0,None,4.5,[],[],[],22


In [6]:
results.sample(1)

,team,mask,current,questionsTotal,synchRequest,position,controversials,flags,teamMembers,tournament_id
395832,"{'id': 39997, 'name': 'Эталон этанола', 'town'...",11001100011101000000100X100111000101,"{'name': 'Эталон этанола', 'town': {'id': 197,...",15.0,"{'id': 58990, 'venue': {'id': 3112, 'name': 'М...",421.0,"[{'id': 94352, 'questionNumber': 23, 'answer':...",[],"[{'flag': 'Б', 'usedRating': 8405, 'rating': 8...",4986


In [7]:
# посмотрим как выглядят примеры записей в таблице

results.loc[166992, 'current'], results.loc[166992, 'team'], results.loc[166992, 'teamMembers'], results.loc[166992, 'synchRequest']

({'name': 'Южный Парк', 'town': {'id': 236, 'name': 'Одесса'}},
 {'id': 4777, 'name': 'Южный Парк', 'town': {'id': 236, 'name': 'Одесса'}},
 [{'flag': 'К',
   'usedRating': 4598,
   'rating': 4602,
   'player': {'id': 26243,
    'name': 'Николай',
    'patronymic': 'Алексеевич',
    'surname': 'Пручковский'}},
  {'flag': 'Б',
   'usedRating': 3612,
   'rating': 4337,
   'player': {'id': 35920,
    'name': 'Владимир',
    'patronymic': 'Викторович',
    'surname': 'Шевчук-Иркмаан'}},
  {'flag': 'Б',
   'usedRating': 2771,
   'rating': 4160,
   'player': {'id': 67902,
    'name': 'Дмитрий',
    'patronymic': 'Леонидович',
    'surname': 'Адамовский'}},
  {'flag': 'Л',
   'usedRating': 1791,
   'rating': 3584,
   'player': {'id': 96079,
    'name': 'Ирина',
    'patronymic': 'Вячеславовна',
    'surname': 'Горавская'}}],
 {'id': 10653, 'venue': {'id': 3134, 'name': 'Одесса'}})

In [8]:
results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 441410 entries, 2407 to 528742
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   team            441410 non-null  object 
 1   mask            441410 non-null  object 
 2   current         441410 non-null  object 
 3   questionsTotal  441410 non-null  float64
 4   synchRequest    319488 non-null  object 
 5   position        441410 non-null  float64
 6   controversials  441410 non-null  object 
 7   flags           441410 non-null  object 
 8   teamMembers     441410 non-null  object 
 9   tournament_id   441410 non-null  int64  
dtypes: float64(2), int64(1), object(7)
memory usage: 53.2+ MB


Турниры

In [9]:
tournaments = pd.DataFrame.from_dict(tournaments_raw, orient='index')
tournaments['dateStart'] = tournaments['dateStart'].str[:-6]  # remove timezones coz it's not so important here
tournaments['dateEnd'] = tournaments['dateEnd'].str[:-6]

tournaments['dateStart'] = pd.to_datetime(tournaments['dateStart'], format='%Y-%m-%d %H:%M:%S')
tournaments['dateEnd'] = pd.to_datetime(tournaments['dateEnd'], format='%Y-%m-%d %H:%M:%S')

tournaments.head()

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
1,1,Чемпионат Южного Кавказа,2003-07-25,2003-07-27,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
2,2,Летние зори,2003-08-09,2003-08-09,"{'id': 2, 'name': 'Обычный'}",/seasons/1,[],None,None
3,3,Турнир в Ижевске,2003-11-22,2003-11-24,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
4,4,Чемпионат Украины. Переходной этап,2003-10-11,2003-10-12,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None
5,5,Бостонское чаепитие,2003-10-10,2003-10-13,"{'id': 2, 'name': 'Обычный'}",/seasons/2,[],None,None


In [10]:
tournaments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5528 entries, 1 to 6485
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            5528 non-null   int64         
 1   name          5528 non-null   object        
 2   dateStart     5528 non-null   datetime64[ns]
 3   dateEnd       5528 non-null   datetime64[ns]
 4   type          5528 non-null   object        
 5   season        5434 non-null   object        
 6   orgcommittee  5528 non-null   object        
 7   synchData     1855 non-null   object        
 8   questionQty   4343 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 431.9+ KB


In [11]:
train_test_tournaments = tournaments[tournaments['dateStart'].dt.year.isin([2019, 2020])]

Игроки

In [12]:
players = pd.DataFrame.from_dict(players_raw, orient='index')
players.head()

,id,name,patronymic,surname
1,1,Алексей,None,Абабилов
10,10,Игорь,,Абалов
11,11,Наталья,Юрьевна,Абалымова
12,12,Артур,Евгеньевич,Абальян
13,13,Эрик,Евгеньевич,Абальян


In [13]:
players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204063 entries, 1 to 224704
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          204063 non-null  int64 
 1   name        204063 non-null  object
 2   patronymic  204032 non-null  object
 3   surname     204063 non-null  object
dtypes: int64(1), object(3)
memory usage: 7.8+ MB


## Преобразуем результаты в повопросные результаты для каждого игрока

In [14]:
print(results.shape)
results = results[results['tournament_id'].isin(train_test_tournaments['id'])]
print(results.shape)


(441410, 10)
(108894, 10)


In [15]:
# filter results of tournaments in 2019, 2020 only
results = results[results['tournament_id'].isin(train_test_tournaments['id'])]

# construct player_results DataFrame for each player
results['number_of_players'] = results['teamMembers'].apply(lambda team: len(team))
results['team'] = results['team'].apply(lambda x: x['id'])
results = results[results['number_of_players'] > 0]
count = results['number_of_players']  # the trick below used for transform we use at work, hope it will be clear
count = count.repeat(count.max())
count.index = count.index.rename('old_index')
count = count.reset_index()
count['count'] = count.groupby('old_index').cumcount()
count = count[count['count'] < count['number_of_players']].drop(['number_of_players'], axis=1)
count.set_index('old_index', inplace=True)

player_results = results.join(count)
player_results['count'] = player_results['count'].astype('int')
player_results['player'] = player_results.apply(lambda row: row['teamMembers'][row['count']], axis=1)
player_results = pd.concat([
    player_results.drop(['player'], axis=1).reset_index(drop=True), 
    pd.DataFrame(player_results['player'].tolist())
], axis=1)
player_results['player'] = player_results['player'].apply(lambda x: x['id'])
player_results.drop(['count', 'current', 'synchRequest', 'controversials', 'flag', 'flags', 'teamMembers'], axis=1, inplace=True)

In [16]:
player_results.head()

,team,mask,questionsTotal,position,tournament_id,number_of_players,usedRating,rating,player
0,45556,111111111011111110111111111100010010,28.0,1.0,4772,6,13507,13507,6212
1,45556,111111111011111110111111111100010010,28.0,1.0,4772,6,10988,13185,18332
2,45556,111111111011111110111111111100010010,28.0,1.0,4772,6,8534,12801,18036
3,45556,111111111011111110111111111100010010,28.0,1.0,4772,6,6401,12801,22799
4,45556,111111111011111110111111111100010010,28.0,1.0,4772,6,4252,12757,15456


In [17]:
# construct player_results DataFrame for each question
# drop columns that are not required to decrease amount of memory used 

player_results['questions_count'] = player_results['mask'].apply(lambda x: len(x))
player_results = player_results[player_results['questions_count'] > 0]
count = player_results['questions_count']
count = count.repeat(count.max())
count.index = count.index.rename('old_index')
count = count.reset_index()
count['count'] = count.groupby('old_index').cumcount()
count = count[count['count'] < count['questions_count']].drop(['questions_count'], axis=1)
count.set_index('old_index', inplace=True)

question_results = player_results.join(count)
question_results['count'] = question_results['count'].astype('int')
question_results['result'] = question_results.apply(lambda row: row['mask'][row['count']], axis=1)

tournament_id + count - уникальный ключ каждого вопроса

In [18]:
train_tournaments = tournaments[tournaments['dateStart'].dt.year == 2019]
test_tournaments = tournaments[tournaments['dateStart'].dt.year == 2020]

train = question_results[question_results['tournament_id'].isin(train_tournaments['id'])]
test = question_results[question_results['tournament_id'].isin(test_tournaments['id'])]

In [19]:
set(train['tournament_id']) & set(test['tournament_id'])

set()

In [20]:
len(set(train['player']) & set(test['player'])), len(set(train['player']))

(24614, 59101)

турниры не просочинились между годами - это хорошо.

# Baseline models 

Построим разреженную матрицу для вопросов и игроков, таргетом в которой будет булева величина - ответил игрок на вопрос или нет. 

In [21]:
train = train[train['result'].isin(['1', '0'])]
X_train = train[['tournament_id', 'player', 'count']]
y_train = train['result'].astype('int')

X_train, y_train

(        tournament_id  player  count
 0                4772    6212      0
 0                4772    6212      1
 0                4772    6212      2
 0                4772    6212      3
 0                4772    6212      4
 ...               ...     ...    ...
 552093           6255  217156     31
 552093           6255  217156     32
 552093           6255  217156     33
 552093           6255  217156     34
 552093           6255  217156     35
 
 [20910740 rows x 3 columns],
 0         1
 0         1
 0         1
 0         1
 0         1
          ..
 552093    0
 552093    0
 552093    0
 552093    0
 552093    0
 Name: result, Length: 20910740, dtype: int64)

Закодируем вопросы и игроков с помощью OneHotEncoder

In [22]:
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
import numpy as np

encoder_questions = OneHotEncoder(handle_unknown='ignore')
encoder_players = OneHotEncoder(handle_unknown='ignore')

questions_ohe = encoder_questions.fit_transform(X_train[['tournament_id', 'count']])
players_ohe = encoder_players.fit_transform(X_train[['player']])

X_train = hstack((questions_ohe, players_ohe))
X_train.shape

(20910740, 60276)

In [23]:
from sklearn.linear_model import LogisticRegression
import pickle

TRAIN = False

baseline = LogisticRegression(max_iter=10000, n_jobs=-1)
if TRAIN:
    baseline.fit(X_train, y_train)
    with open('baseline.pickle', 'wb') as bf:
        pickle.dump(baseline, bf)
else:
    with open('baseline.pickle', 'rb') as bf:
        baseline = pickle.load(bf)

In [24]:
baseline_coef = baseline.coef_
baseline_coef


array([[-0.0620721 ,  0.22531979,  0.19548669, ..., -0.48117916,
         0.75720149,  1.17367353]])

## Ранжируем команды

Задача построить DF с колонками team, tournament -> position для тестирования модели.

Будем использовать коэффициенты линейной регресии, для ранжирования команды. Можно попробовать ставить в соответствие команде:
* максимальный коэффициент
* средний коэффициент участников

Затем ранжировать команды по этому коэффициенту.

Хотя коэффициенты линейной регрессии могут для разных игроков быть разных порядков. Предполагаю, что для игроков одного класса они будут близки, поскольку они отвечают на вопросы одинаковой сложности.

In [25]:
players_coef = baseline_coef[questions_ohe.shape[1]:]

test = test[test['result'].isin(['1', '0'])]
y_test = test['result'].astype(int)
X_test = encoder_players.transform(test[['player']])

y_test.shape, X_test.shape, players_coef.shape

((4469664,), (4469664, 59101), (0, 60276))

In [26]:
test['player_coef'] = pd.Series((X_test @ baseline_coef[:, questions_ohe.shape[1]:].T)[:, 0])
test.sample(5)

,team,mask,questionsTotal,position,tournament_id,number_of_players,usedRating,rating,player,questions_count,count,result,player_coef
511173,46916,100110111110110111010110101101011111,25.0,16.0,6171,6,7067,10600,134506,36,29,1,1.341036
518552,50435,011100000111101110110000111101101000,19.0,87.5,6193,6,985,5908,117722,36,0,0,0.403266
411443,66401,000010100010000000101010010000101000,9.0,351.5,5856,2,53,53,175756,36,17,0,1.937677
297192,72798,101000001001000011000000110100000101,11.0,881.0,5754,5,2762,2762,209306,36,24,1,-0.990214
564372,60732,000000000000000000000011000000000000,2.0,4.0,6410,4,686,1029,161570,36,8,0,1.305088


Подход к ранжированию через максимум

In [27]:
agg_test_max = test.groupby(['team', 'tournament_id', 'position']).agg({'player_coef': 'max'})
agg_test_max.sort_values(by='player_coef', ascending=False, inplace=True)
agg_test_max['prediction'] = agg_test_max.groupby('tournament_id')['player_coef'].cumcount()
agg_test_max['prediction'] = agg_test_max.groupby(['tournament_id', 'player_coef']).transform('mean')
agg_test_max.reset_index(inplace=True)

In [28]:
from scipy.stats import spearmanr, kendalltau

x = agg_test_max['position']
y = agg_test_max['prediction']

# Расчет статистики Спирмена
corr_spearman, p_value = spearmanr(x, y)
print("Коэффициент Спирмена:", corr_spearman)
print("p-значение:", p_value)

# Расчет статистики Кендалла
corr_kendall, p_value = kendalltau(x, y)
print("Коэффициент Кендалла:", corr_kendall)
print("p-значение:", p_value)


Коэффициент Спирмена: 0.6350996322539653
p-значение: 0.0
Коэффициент Кендалла: 0.4642982299976038
p-значение: 0.0


Подход к ранжированию через минимум

In [29]:
agg_test_mean = test.groupby(['team', 'tournament_id', 'position']).agg({'player_coef': 'mean'})
agg_test_mean.sort_values(by='player_coef', ascending=False, inplace=True)
agg_test_mean['prediction'] = agg_test_mean.groupby('tournament_id')['player_coef'].cumcount()
agg_test_mean['prediction'] = agg_test_mean.groupby(['tournament_id', 'player_coef']).transform('mean')
agg_test_mean.reset_index(inplace=True)

In [30]:
from scipy.stats import spearmanr, kendalltau

x = agg_test_mean['position']
y = agg_test_mean['prediction']

# Расчет статистики Спирмена
corr_spearman, p_value = spearmanr(x, y)
print("Коэффициент Спирмена:", corr_spearman)
print("p-значение:", p_value)

# Расчет статистики Кендалла
corr_kendall, p_value = kendalltau(x, y)
print("Коэффициент Кендалла:", corr_kendall)
print("p-значение:", p_value)


Коэффициент Спирмена: 0.633642026429817
p-значение: 0.0
Коэффициент Кендалла: 0.4631452358299374
p-значение: 0.0


# EM алгоритм

In [31]:
train.head()

,team,mask,questionsTotal,position,tournament_id,number_of_players,usedRating,rating,player,questions_count,count,result
0,45556,111111111011111110111111111100010010,28.0,1.0,4772,6,13507,13507,6212,36,0,1
0,45556,111111111011111110111111111100010010,28.0,1.0,4772,6,13507,13507,6212,36,1,1
0,45556,111111111011111110111111111100010010,28.0,1.0,4772,6,13507,13507,6212,36,2,1
0,45556,111111111011111110111111111100010010,28.0,1.0,4772,6,13507,13507,6212,36,3,1
0,45556,111111111011111110111111111100010010,28.0,1.0,4772,6,13507,13507,6212,36,4,1


In [ ]:
from scipy.sparse import coo_array
import torch 

X_train = coo_array(X_train)

indices = np.vstack((X_train.row, X_train.col))

X_train_torch = torch.sparse.FloatTensor(
    torch.LongTensor(indices), 
    torch.FloatTensor(X_train.data), 
    torch.Size(X_train.shape)
).to_dense()

y_train_torch = torch.from_numpy(y_train)

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [59]:
import torch


# indices = np.vstack((train_sparse.row, train_sparse.col))

train_torch = torch.FloatTensor(X_train)

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [42]:
np.where(unique_teams[0] == teams)[0][0]

1213485

In [39]:
teams[i]

0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
0    45556
Name: team, dtype: int64